# 화재유형별 추출
* 미리 저장해둔 ./data/index.csv를 호출하여 사용합니다.
* index.csv에는 index.xhtml에서 추출한 목차와 임의로 설정한 화재유형컬럼이 있습니다.

* 화제유형컬럼을 통해 연관된 자료를 한번에 호출하는 코드입니다.
* 콤보박스 리스트를 통해 데이터를 추가하고 리스트 위젯을 통해 정리한 뒤 저장하시면 됩니다.
* 저장한 파일은 data폴더에 저장됩니다.

In [1]:
#!/usr/bin/env python
# coding: utf-8

from PyQt5 import uic
from PyQt5.QtWidgets import *
from urllib import request
from bs4 import BeautifulSoup
import pandas as pd
import os
import sys
import torch

from pytorch_pretrained_bert.file_utils import PYTORCH_PRETRAINED_BERT_CACHE
from pytorch_pretrained_bert.modeling import BertForSequenceClassification
from src_tokenizer.tokenization import BertTokenizer # 수정
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
from tqdm import tqdm, trange
import numpy as np
import sys
sys.path.append('C:/Users/Admin/.conda/envs/pytoch/Lib/site-packages')

data_dir = './fire'
task_name = 'cola'
vocab_file = 'vocab.korean.rawtext.list'
bert_model_path = './output'
cache_dir = ""
max_seq_length = 128
eval_batch_size = 8
local_rank = -1
seed = 42
gradient_accumulation_steps = 1
no_cuda = False
device = torch.device("cuda" if torch.cuda.is_available() and not no_cuda else "cpu")
label_list = ['0', '1', '2', '3', '4', '5']
tokenizer = BertTokenizer.from_pretrained(vocab_file, do_lower_case=False)

cache_dir = cache_dir if cache_dir else os.path.join(str(PYTORCH_PRETRAINED_BERT_CACHE), 'distributed_{}'.format(local_rank))
model = BertForSequenceClassification.from_pretrained(bert_model_path,
              cache_dir=cache_dir,
              num_labels = 6) # num_labels_task 바꿔야함
model.to(device)

path = os.getcwd().replace('\\', '/')

# 저장된 xhtml를 호출
url_1 = "/Daejeon/index.xhtml"
url = 'file:///' + path + url_1

with request.urlopen(url) as f:
    html = f.read().decode('utf-8')
bs = BeautifulSoup(html, 'html.parser')

page_1 = bs.select_one('body > .Section.Section-2.Paper > .HeaderPageFooter > .Page')

# 한줄장 하나의 Nomal 클래스를 가지는 p에 저장됨

global context
context = page_1.select('.Normal')

# UI 연결
form_class = uic.loadUiType('./ui/hwp_parser_2.ui')[0]


# In[2]:
# 화면을 띄우는데 사용되는 class 선언
class WindowClass(QMainWindow, form_class):
    def __init__(self):
        super().__init__()
        self.setupUi(self)

        # 데이터 추출에서 사용할 인덱스 데이터 프레임
        self.index =  pd.read_csv('./data/index.csv',encoding = 'euc-kr')
        
        # 추출할 문장을 저장할 공간
        self.sentence_list = []


############### 시그널 관리

        # 버튼에 기능을 연결
        self.pushButton.clicked.connect(self.button1Function)
        self.pushButton_2.clicked.connect(self.button2Function)
        self.pushButton_3.clicked.connect(self.button3Function)
        self.pushButton_4.clicked.connect(self.button4Function)
        # text 조작


        # 리스트위젯 조작
        self.listWidget.itemClicked.connect(self.listClickFunction)

############### 시그널 관리

############### 저장
    # btn_1: 리스트의 항목을 저장
    def button1Function(self):
        with open('./data/test.txt', 'w', encoding='utf-8') as f:
            for i in self.sentence_list:
                for j in i:
                    f.write(str(j))
                f.write('\n')


############## 저장

############## text 예측 관리
    # 예측버튼 누르면 시작

    def button4Function(self):
        #text 창의 text를 tokens_a에 저장
        tokens_a = self.textEdit.toPlainText()
        
        # bert 모델을 통해 분류
        idx = self.pre_text(tokens_a)
        
        # 분류모델의 결과값을 통해 index에서 목차번호 추출
        dict = {0:'기타',1:'건물',2:'차량',3:'임야',4:'선박',5:'위험물'}
        select = self.index[self.index['5'] == dict[idx]]['2']

        # 찾고싶은 데이터의 index를 추가
        input_index_list = [ self.index[self.index['2'] == i].index[0] for i in select]


        # 추출할 데이터의 index의 범위를 설정

        for idx in input_index_list:
            for i in context:
                if i.text.find(self.index.iloc[idx, 2]) == 0:
                    self.s_idx = context.index(i)
                if idx == 133:
                    if i.text.find('붙    임') == 0:
                        self.e_idx = context.index(i)
                        break
                else:
                    if i.text.find(self.index.iloc[idx + 1, 2]) == 0:
                        self.e_idx = context.index(i)
                        break

            # 데이터를 리스트위젯에 시각화
            self.listWidget.addItem(self.index.iloc[idx , 4])

            # 데이터를 find_context함수를 통해서 추출 후 추가

            self.sentence_list.append(self.find_context())

    # 선택된 항목의 데이터를 추출 후 반환
    def find_context(self):
        sentence = ''
        count = 0
        full_sentence = []

        for i in range(self.s_idx - 1, self.e_idx):
            content = context[i]
            # context[i]가 테이블일 시 테이블 형식으로 저장
            if content.select('table'):
                # 기존 텍스트를 먼저 저장 후 리셋
                full_sentence.append(sentence)
                sentence = ''
                
                # 테이블이 text box로 사용된 경우
                if (len(content.select('td')) == 1)&(len(content.select('tr')) == 1):
                    # 테이블 데이터를 저장
                    table =[ i.text for i in content.select('p') ]
        
                # 테이블로 사용된 겨우
                else:
                    table = []
            
                    for i in content.select('tr'):
                        td_list = []
                        if i.text != '':
                            # 일반적인 테이블
                            if len(i.select('td')) != 1:
                                for j in i.select('td'):
                                    if j.text != '':
                                        td_list.append(j.text)
                                table.append(td_list)
                            
                            # text box로 사용되었으나 여러개의 row가 지정된 경우
                            else:
                                for z in i.select('p'):
                                    td_list.append(z.text)
                                table.append(td_list)
        

                # 테이블 데이터 안의 p의 개수를 파악
                count = len(content.select('p'))
        
        
                # 테이블 저장 후 리셋
                full_sentence.append(table)
                full_sentence.append('\n')
                table = ''

            # context[i]가 테이블일 시 테이블 형식으로 저장
            else:
                # table 파트에서 받은 테이블 안의 p의 개수를 통해 테이블 p를 생략
                if count != 0:
                    count -= 1
                else:
                    # table p를 넘긴 후 텍스트 p를 추가
                    sentence += content.text.strip() 
            if i == self.e_idx - 1:
                full_sentence.append(sentence)
        return full_sentence
    
    # bert모델을 통해 text의 유형을 분류
    def pre_text(self, tokens_a):
        tokens_a = tokenizer.tokenize(tokens_a)
        if len(tokens_a) > max_seq_length - 2:
            tokens_a = tokens_a[:(max_seq_length - 2)]  # max_seq_length에 맞게 길이를 조절
        tokens = ["[CLS]"] + tokens_a + ["[SEP]"]
        segment_ids = [0] * len(tokens)  # 문장이 2개 이어진게 아니기 때문에 문장구분을 안해도됨
        input_ids = tokenizer.convert_tokens_to_ids(tokens)  # 인덱싱
        input_mask = [1] * len(input_ids)
        padding = [0] * (max_seq_length - len(input_ids))
        input_ids += padding
        input_mask += padding
        segment_ids += padding
        label_id = 3
        # input data의 길이가 지정된 길이와 동일하게 고정
        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length

        # 전처리
        # input하고 싶은 데이터를 전처리해주는 함수를 봐야함 convert_examples_to_features 분해
        all_input_ids = torch.tensor([input_ids], dtype=torch.long)
        all_input_mask = torch.tensor([input_mask], dtype=torch.long)
        all_segment_ids = torch.tensor([segment_ids], dtype=torch.long)
        all_label_ids = torch.tensor([label_id], dtype=torch.long)




        eval_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
        eval_sampler = SequentialSampler(eval_data)
        eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=eval_batch_size)


        # 평가모드
        model.eval()

        for input_ids, input_mask, segment_ids, label_ids in tqdm(eval_dataloader, desc="Evaluating"):
            input_ids = input_ids.to(device)
            input_mask = input_mask.to(device)
            segment_ids = segment_ids.to(device)

        with torch.no_grad():

            logits = model(input_ids, segment_ids, input_mask)

        logits = logits.detach().cpu().numpy()
        current_out = np.argmax(logits, axis=1)
        pre = 0
        # 결과값
        for key in current_out:
            pre = int(key)


        return pre



############## 콤보 박스 관리

############## 리스트 위젯 관리
    # 선택한 리스트의 인덱스를 호출
    def listClickFunction(self):
        self.delete_index = self.listWidget.currentRow()

    # btn_2: 리스트에서 선택한 항목을 삭제
    def button2Function(self):
        try:
            self.listWidget.takeItem(self.delete_index)
            del self.sentence_list[self.delete_index]
        except:
            pass

    #btn_3 : 모든 항목을 제거
    def button3Function(self):
        self.listWidget.clear()
        self.sentence_list = []

############## 리스트 위젯 관리





if __name__ == '__main__':
    # QApplication : 프로그램을 실행시켜주는 클래스

    app = QApplication(sys.argv)

    # WindowClass 인스턴스 생성
    mywindow = WindowClass()

    # 프로그램 화면을 보여주는 코드
    mywindow.show()

    # 프로그램을 이벤트루프로 진입시키는 코드
    app.exec_()

Evaluating: 100%|███████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 250.26it/s]
